Members :
- Jiren REN : renjiren120@gmail.com
- Xi WANG : lilaswang2227@gmail.com

## Implementing the BPE algorithm
The aim of this exercize is to implement the BPE tokenization algorithm. As a reminder, the principle consists in gathering the words or “tokens” that appear the most times in succession.

For example, if we consider the corpus containing the words in the following table (with the number of occurrences of each word):

| words | occurrence |
|------|-----------|
| voting | 2 |
| vote | 3 |
| slow | 1 |
| slowly | 2 |


And if the initial “tokens” are the letters of the alphabet, then the prefix “vo” will initially be added to the list of sub-words (tokens), cause the bigram "v" "o" occurs 5 times (2 + 3).

The steps involved in implementing the BPE algorithm are as follows:
1. Download a text corpus (here a wikipedia page)
2. Cut the text into words (using the “space” and “ponctuation” characters) and count the number of occurrences of each word.
3. Initialize the word dictionary with the initial tokens (letters of the alphabet)
4. Run BPE algorithm (learn vocabulary)
5. Test token decomposition on selected sentences (apply learned rules)


In the documents you will observe a lot of TODO in the code, replace it by your code.

### Submit your report

You can use the notebook as submission, you will also need to comment your experiments/choices !!! 

### Step 1: Download a corpus

In [5]:
from urllib.request import urlopen, Request
import re
import numpy as np
import collections
import json
url_request  = 'https://fr.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&explaintext&redirects=1&titles=Gr%C3%A8ce_antique'
wikipedia_request = Request(url_request)
wikipedia_request.add_header("User-Agent", "Course (thomas.gerald@lisn.fr)")

raw_page = urlopen(wikipedia_request)
json_page = json.load(raw_page)


In [6]:
corpus = list(json_page['query']['pages'].values())[0]['extract']

### Step 2: Splitting text into words

To split the text into words, we'll use the following regex ```r'(\b[^\s]+\b)'```. To count words, we'll use python's Counter object. 
1. Store each word and its number of occurrences in **count_words**.
2. Give the 10 most frequent words (you'll store them in most_commons_words).

In [ ]:
from collections import Counter


word_regex = re.compile(r'(\b[^\s]+\b)')
words = word_regex.findall(corpus)

count_words = Counter(words) # TODO
most_commons_words = count_words.most_common(10) # TODO
print(count_words)
print(most_commons_words)

Counter({'de': 1694, 'la': 1101, 'et': 978, 'des': 929, 'les': 773, 'à': 601, 'le': 519, 'en': 481, 'qui': 380, 'du': 352, 'dans': 351, 'une': 349, 'est': 313, 'un': 313, 'par': 272, 'sont': 272, 'se': 234, 'plus': 223, 'Les': 193, 'pour': 188, 'sur': 179, 'que': 170, 'au': 164, 'grecque': 159, 'La': 154, 'ou': 153, 'J.-C': 151, 'av': 149, 'grec': 148, "l'époque": 138, 'comme': 131, 'avec': 129, 'leur': 128, 'monde': 126, 'Grèce': 122, 'mais': 122, 'siècle': 110, 'cités': 110, 'aux': 109, 'ce': 109, 'a': 108, 'pas': 105, 'aussi': 104, 'ont': 103, 'son': 98, 'Le': 97, 'durant': 89, 'même': 88, 'antique': 84, 'ne': 82, 'Grecs': 78, 'il': 76, 'culture': 74, 'hellénistique': 72, 'cité': 71, 'entre': 70, "d'un": 70, 'grecques': 66, 'classique': 66, 'période': 66, 'notamment': 65, "d'une": 64, 'souvent': 63, 'cette': 63, 'être': 63, 'politique': 62, 'peut': 58, 'antiques': 54, 'surtout': 53, 'forme': 49, 'bien': 49, 'Athènes': 49, 'dont': 48, 'alors': 48, 'partir': 47, 'archaïque': 47, 'sous

### Notes (tokenization choices)
We use the provided regular expression `(\b[^\s]+\b)` to split the Wikipedia excerpt into “words”.
Here, `[^\s]+` captures contiguous non-whitespace spans, while `\b...\b` (word boundaries) usually excludes punctuation at the edges (since punctuation is typically non-word characters).
This tokenization preserves case, so tokens like `de` and `De` are counted as different words. We keep case information (e.g., proper nouns and sentence-initial words).
We also check a few examples (e.g., `d'une`, `être`, `l'Antiquité`) to verify how apostrophes/accents are handled by this regex.

### Step 3: Initialize word dictionary with initial tokens (letters of the alphabet)

Create the initial vocabulary in the vocab variable. How many initial tokens do you have?

In [ ]:
vocab = set() # TODO
for w in count_words.keys():
    vocab.update(list(w))
print("Initial token count:", len(vocab))

Initial token count: 105


### Step 4: Learning the tokenizer
To learn the tokenizer we need several functions:
1. A function to calculate the frequency of each token pair.
2. A function to merge a pair

Several variables will be required:
1. **vocab** containing current vocabulary
2. **merge_rules** containing all the merge rules (a dictionary containing as key a pair of tokens to merge and the result of the token merge). For example: {('e', 's'), 'es', ('en', 't') :'ent'}.
3. **splits** A dictionary containing the current breakdown of the corpus, with the word as key and the list of “tokens” as value.


In [ ]:
# In the first step splits contains the words broken down into characters
splits = {w: [c for c in w] for w in count_words.keys()} # TODO
{k: splits[k] for k in list(splits.keys())[:5]}

{'La': ['L', 'a'],
 'Grèce': ['G', 'r', 'è', 'c', 'e'],
 'antique': ['a', 'n', 't', 'i', 'q', 'u', 'e'],
 'est': ['e', 's', 't'],
 'une': ['u', 'n', 'e']}

#### Compute the frequency of token pairs
Create a function **compute_pair_freqs**, which, given the words broken down into tokens (splits dictionary) and the frequency of the words, returns the frequency of each pair of tokens (note only successive sub-words).

In [ ]:
def compute_pair_freqs(splits, word_freqs):
    pair_freqs = {}
    for word, freq in word_freqs.items(): # TODO
        tokens = splits[word]
        if len(tokens) < 2:
            continue
        for i in range(len(tokens) - 1):
            pair = (tokens[i], tokens[i+1])
            if pair not in pair_freqs:
                pair_freqs[pair] = 0
            pair_freqs[pair] += freq
    return pair_freqs

In [39]:
pair_freqs = compute_pair_freqs(splits, count_words)
{k: pair_freqs[k] for k  in list(pair_freqs.keys())[:5]}

{('L', 'a'): 165,
 ('G', 'r'): 255,
 ('r', 'è'): 244,
 ('è', 'c'): 270,
 ('c', 'e'): 1163}

#### Find the most frequent pair and merge a pair
1. Create a function **most_frequent(pair_freqs)** returning the most frequent pair of tokens.
2. Create a **merge_pair()** function which, given a pair, returns the new splits of the corpus.

In [ ]:
def most_frequent(pair_freqs):
    return max(pair_freqs.items(), key=lambda x: x[1]) # TODO
most_frequent(pair_freqs)

(('e', 's'), 5663)

In [49]:
def merge_pair(a : str, b: str, splits):
    '''
        splits : the dataset of words tokenized
        a : the first token  
        b : the second tokens

        return : splits with a,b tokens merged
    '''
    for word in splits.keys():
        tokens = splits[word] # TODO
        if len(tokens) < 2:
            continue
        i = 0
        while i < len(tokens)-1:
            if tokens[i] == a and tokens[i+1] == b:
                tokens = tokens[:i] + [a + b] + tokens[i+2:]
            else:
                i += 1
        splits[word] = tokens
    return splits

In [50]:
new_splits = merge_pair(*most_frequent(pair_freqs)[0], splits)
print(new_splits['grecques'])

['g', 'r', 'e', 'c', 'q', 'u', 'es']


#### Apply the algorithm until the desired vocabulary size is reached.
Create a BPE object that takes as arguments a corpus, a vocabulary size and train the BPE algorithm. The algorithm stores the final vocabulary in the **vocabulary** attribute and the merge rules in **merge_rules**.
For merge_rules, here's an example of its contents:
```
{('e', 's'): 'es',
 ('n', 't'): 'nt',
 ('q', 'u'): 'qu',
 ('r', 'e'): 're',
 ('o', 'n'): 'on',
 ('d', 'e'): 'de',
 ('l', 'e'): 'le',
 ('t', 'i'): 'ti',
 ('l', 'a'): 'la',
 ('i', 's'): 'is',
 ('e', 'nt'): 'ent', ...
 }
```

In [56]:
class BPE:
    def __init__(self, corpus, vocabulary_size=500):
        super().__init__()
        self.word_regex = re.compile(r'(\b[^\s]+\b)')
        words = self.word_regex.findall(corpus)

        # counting words
        count_words = collections.Counter(words)
        # create initial vocab
        self.vocab = list({char for word in count_words.keys() for char in word })
        self.vocab.sort()
        # create the initial split
        splits = {word: [c for c in word] for word in count_words.keys()}
        # initialise merge_rules
        self.merge_rules = {}
        self.stop_reason = None
        while len(self.vocab) < vocabulary_size: # TODO
            pair_freqs = compute_pair_freqs(splits, count_words)
            if not pair_freqs:
                self.stop_reason = "Stopped: no more mergeable pairs (all token sequences have length < 2)."
                break

            best_pair, best_freq = most_frequent(pair_freqs)  # best_pair is (a,b)
            a, b = best_pair
            splits = merge_pair(a, b, splits)
            merge_token = a + b
            self.merge_rules[best_pair] = merge_token
            self.vocab.append(merge_token)

        if self.stop_reason is None:
            self.stop_reason = f"Stopped: reached target vocabulary_size={vocabulary_size}."
    
    def tokenize(self, text):
        words = self.word_regex.findall(text)
        splits = [[l for l in word] for word in words]
        for pair, merge in self.merge_rules.items():
            for idx, split in enumerate(splits):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2 :]
                    else:
                        i += 1
                splits[idx] = split
    
        return sum(splits, [])


In [57]:
my_bpe = BPE(corpus)

In [58]:
print("First 10 merge rules:", [f"{p}->{m}" for p, m in list(my_bpe.merge_rules.items())[:10]])
print("Training stopped because:", my_bpe.stop_reason)

First 10 merge rules: ["('e', 's')->es", "('n', 't')->nt", "('q', 'u')->qu", "('r', 'e')->re", "('o', 'n')->on", "('d', 'e')->de", "('l', 'e')->le", "('t', 'i')->ti", "('l', 'a')->la", "('i', 's')->is"]
Training stopped because: Stopped: reached target vocabulary_size=500.


In [59]:
texte = '''culture grecques développée en Grèce '''
my_bpe.tokenize(texte)[:12]

['culture', 'grecques', 'dévelop', 'p', 'ée', 'en', 'Grèce']

### Observations
The first learned merge rules correspond to very frequent French character bigrams, such as "('e', 's')->es", "('n', 't')->nt", and common function-word patterns like "('o', 'n')->on", "('d', 'e')->de", "('l', 'e')->le".
This suggests BPE quickly captures common orthographic sequences that appear across many words.

Training stopped because the target vocabulary size was reached ('vocabulary_size=500'). In practice, it may still be possible to merge additional pairs, but the final subword vocabulary is explicitly controlled by this hyperparameter.

On the test sentence *"culture grecques développée en Grèce"*, the tokenizer keeps frequent words intact (e.g., 'culture', 'grecques', 'en', 'Grèce') while splitting rarer or morphologically complex forms into subword units (e.g., 'dévelop' + 'p; + 'ée').
This illustrates the main benefit of BPE compared to word-level tokenization: rare or unseen words can be represented as combinations of known subwords instead of becoming out-of-vocabulary tokens.

#### Test by modifying parameters or corpus
Test the algorithm with different hyper-parameters or data

## Using sentencepiece
We're now going to use the `sentencepiece` library, which can be installed (if not already installed) with pip : 

`! pip install sentencepiece`

To train the tokenizer, we'll use the `train` function of `SentencePieceTrainer`. 

In [34]:
! pip install sentencepiece

In [60]:
import sentencepiece as spm

with open("test-cours.txt", "w") as f:
    f.write(corpus)
spm.SentencePieceTrainer.train(input="test-cours.txt", model_type='BPE',  model_prefix='m', vocab_size=500)

Looking at the “m.vocab” file, what are the differences with the vocabulary learned with your implementation? What modifications could you consider to obtain a similar vocabulary?

In [61]:
import pandas as pd

def read_sp_vocab(path="m.vocab", n_head=30, n_tail=30, n_sample=20, seed=0):
    # SentencePiece vocab format: <piece>\t<score>
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.rstrip("\n")
            if not line:
                continue
            # usually tab-separated; sometimes space, so do split(maxsplit=1)
            parts = line.split("\t")
            if len(parts) == 2:
                piece, score = parts[0], parts[1]
            else:
                piece, score = line.split(maxsplit=1)
            rows.append((piece, float(score)))

    df = pd.DataFrame(rows, columns=["piece", "score"])
    df["len"] = df["piece"].str.len()
    df["starts_with_space_marker"] = df["piece"].str.startswith("▁")
    df["is_special"] = df["piece"].isin(["<unk>", "<s>", "</s>", "<pad>"])

    print(f"Total pieces: {len(df)}")
    print(f"Special pieces count: {df['is_special'].sum()}")
    print(f"Pieces starting with '▁' (word-start marker): {df['starts_with_space_marker'].sum()}")
    print(f"Avg piece length: {df['len'].mean():.2f}")

    print("\n--- Head (first lines) ---")
    display(df.head(n_head))

    print("\n--- Tail (last lines) ---")
    display(df.tail(n_tail))

    print("\n--- Sample pieces (random) ---")
    display(df.sample(min(n_sample, len(df)), random_state=seed).sort_values("score", ascending=False))

    print("\n--- Examples: pieces starting with '▁' ---")
    display(df[df["starts_with_space_marker"]].head(20))

    print("\n--- Special tokens (if present) ---")
    display(df[df["is_special"]])

    return df

sp_vocab_df = read_sp_vocab("m.vocab")

Total pieces: 500
Special pieces count: 3
Pieces starting with '▁' (word-start marker): 184
Avg piece length: 3.00

--- Head (first lines) ---


,piece,score,len,starts_with_space_marker,is_special
0,<unk>,0.0,5,False,True
1,<s>,0.0,3,False,True
2,</s>,0.0,4,False,True
3,es,-0.0,2,False,False
4,▁d,-1.0,2,True,False
5,▁l,-2.0,2,True,False
6,nt,-3.0,2,False,False
7,▁p,-4.0,2,True,False
8,qu,-5.0,2,False,False
9,re,-6.0,2,False,False



--- Tail (last lines) ---


,piece,score,len,starts_with_space_marker,is_special
470,H,-467.0,1,False,False
471,É,-468.0,1,False,False
472,V,-469.0,1,False,False
473,z,-470.0,1,False,False
474,â,-471.0,1,False,False
475,ï,-472.0,1,False,False
476,O,-473.0,1,False,False
477,ô,-474.0,1,False,False
478,î,-475.0,1,False,False
479,4,-476.0,1,False,False



--- Sample pieces (random) ---


,piece,score,len,starts_with_space_marker,is_special
15,▁de,-12.0,3,True,False
37,tion,-34.0,4,False,False
90,▁une,-87.0,4,True,False
153,our,-150.0,3,False,False
154,▁J,-151.0,2,True,False
159,and,-156.0,3,False,False
171,od,-168.0,2,False,False
241,aux,-238.0,3,False,False
250,co,-247.0,2,False,False
254,velopp,-251.0,6,False,False



--- Examples: pieces starting with '▁' ---


,piece,score,len,starts_with_space_marker,is_special
4,▁d,-1.0,2,True,False
5,▁l,-2.0,2,True,False
7,▁p,-4.0,2,True,False
10,▁c,-7.0,2,True,False
12,▁s,-9.0,2,True,False
13,▁e,-10.0,2,True,False
14,▁a,-11.0,2,True,False
15,▁de,-12.0,3,True,False
23,▁m,-20.0,2,True,False
25,▁la,-22.0,3,True,False



--- Special tokens (if present) ---


,piece,score,len,starts_with_space_marker,is_special
0,<unk>,0.0,5,False,True
1,<s>,0.0,3,False,True
2,</s>,0.0,4,False,True


### Answer
- Looking at the “m.vocab” file, what are the differences with the vocabulary learned with your implementation? 
1. SentencePiece includes special tokens like `<unk>`, `<s>`, and `</s>` (Special pieces count: 3), while my BPE implementation only merges from the beginning of the character string, without automatically adding these "sentence boundary" and "unknown word" symbols, and lacks an OOV (Out of Context) handling mechanism.
2. SentencePiece encodes word boundaries with the `_` marker: Analysis by m.vocab shows that 184/500 pieces begin with `_` (Pieces starting with `_`: 184), for example, `_d`, `_la`, `_de`, `_une`. SentencePiece explicitly distinguishes "word start" in the vocabulary. It doesn't rely on pre-segmentation but uses `_` to express space/word boundary information. My implementation first uses regex to segment the text into individual "words," then only merges within each word, so the final vocabulary lacks subwords with word boundaries like `_xxx`, thus lacking reversible space information.
3. `m.vocab` stores a score for each piece: Each line of `m.vocab` has a score. This score represents the model's preference/cost for that piece (related to frequency or log probability). My implementation only has "merging rules" and a "vocabulary list," without storing the probability/score for each token, therefore it cannot output "piece + score" like SentencePiece.
4. SentencePiece vocabulary contains punctuation and standalone characters more explicitly. Examples include single-character pieces such as ;, ', /, :, and even numbers 4, 5, 6, 7, 8, 9, and uppercase letters É, À, N. My implementation starts with a character set and can also include these characters; however, because I segment by "word" during training, statistics and merging only occur within words. SentencePiece's learning method based on whole text/space markers makes the vocabulary structure more biased towards "real text fragments" (especially _ prefixes and common function word fragments).
- What modifications could you consider to obtain a similar vocabulary?
1. Add a word-boundary marker like _: During training and encoding, change each word to _ + word, and allow merges containing _. This will learn initial words like _de, _la, _une, making the vocabulary closer to SentencePiece.
2. Train on raw text (no pre-splitting into words), or at least allow merges across word boundaries via the marker: SentencePiece doesn't require pre-splitting. Change my implementation from limiting merges to "intra-word merging" to training on the character sequence of the entire text (using _ to represent spaces). The resulting pairs will be closer to SentencePiece.
3. Include special tokens and handle unknown tokens: Manually add <unk>, <s>, </s> to the vocabulary and support these symbols in tokenize/detokenize.
4. Store token scores: In addition, you can record the frequency of occurrence of each new token obtained from merging, or define a simple score (e.g., -log(freq)) to get an output format similar to piece + score in "m.vocab".

## Detokenization

Propose and implement a methods to detokenize encoded sentence (you should want to extend your vocabulary)

In [68]:
def detokenize_naive(tokens):
    # Naive: just join tokens, cannot reliably restore spaces
    return "".join(tokens)

def detokenize_with_space_marker(tokens, marker="▁"):
    text = "".join(tokens)
    text = text.replace(marker, " ").lstrip()
    return text


class BPE_with_marker:
    def __init__(self, corpus, vocabulary_size=500):
        super().__init__()
        self.word_regex = re.compile(r'(\b[^\s]+\b)')
        words = self.word_regex.findall(corpus)

        # counting words
        count_words = collections.Counter(words)
        # create initial vocab
        self.vocab = list({char for word in count_words.keys() for char in word })
        self.vocab.sort()
        # create the initial split
        splits = {word: [c for c in word] for word in count_words.keys()}
        # initialise merge_rules
        self.merge_rules = {}
        self.stop_reason = None
        while len(self.vocab) < vocabulary_size: # TODO
            pair_freqs = compute_pair_freqs(splits, count_words)
            if not pair_freqs:
                self.stop_reason = "Stopped: no more mergeable pairs (all token sequences have length < 2)."
                break

            best_pair, best_freq = most_frequent(pair_freqs)  # best_pair is (a,b)
            a, b = best_pair
            splits = merge_pair(a, b, splits)
            merge_token = a + b
            self.merge_rules[best_pair] = merge_token
            self.vocab.append(merge_token)

        if self.stop_reason is None:
            self.stop_reason = f"Stopped: reached target vocabulary_size={vocabulary_size}."

    def tokenize_with_marker(self, text, marker="▁"):
        """
        Tokenize with an explicit word-start marker, similar to SentencePiece.
        This makes detokenization easy: marker -> space.
        """
        words = self.word_regex.findall(text)

        # minimal change: prefix each word with marker in the initial split
        splits = [[marker] + list(word) for word in words]

        # apply the SAME merge rules learned during training
        for (a, b), merged in self.merge_rules.items():
            for idx, tokens in enumerate(splits):
                i = 0
                new_tokens = []
                while i < len(tokens):
                    if i < len(tokens) - 1 and tokens[i] == a and tokens[i + 1] == b:
                        new_tokens.append(merged)
                        i += 2
                    else:
                        new_tokens.append(tokens[i])
                        i += 1
                splits[idx] = new_tokens

        # flatten
        return [t for word_tokens in splits for t in word_tokens]

In [69]:
texte = "culture grecques développée en Grèce"

# Version 1: original (no marker)
tokens_v1 = my_bpe.tokenize(texte)
decoded_v1 = detokenize_naive(tokens_v1)

# Version 2: with marker
my_bpe_with_marker = BPE_with_marker(texte)
tokens_v2 = my_bpe_with_marker.tokenize_with_marker(texte, marker="▁")
decoded_v2 = detokenize_with_space_marker(tokens_v2, marker="▁")

print("Original text:         ", texte)
print("\n--- Version 1 (no marker) ---")
print("tokens:", tokens_v1[:30])
print("detokenized:", decoded_v1)

print("\n--- Version 2 (with '▁' marker) ---")
print("tokens:", tokens_v2[:30])
print("detokenized:", decoded_v2)

Original text:          culture grecques développée en Grèce

--- Version 1 (no marker) ---
tokens: ['culture', 'grecques', 'dévelop', 'p', 'ée', 'en', 'Grèce']
detokenized: culturegrecquesdéveloppéeenGrèce

--- Version 2 (with '▁' marker) ---
tokens: ['▁', 'culture', '▁', 'grecques', '▁', 'développée', '▁', 'en', '▁', 'Grèce']
detokenized: culture grecques développée en Grèce


### Conclusion (detokenization)
In the original BPE setup (without an explicit word-boundary symbol), detokenization is ambiguous: once a sentence is encoded as a sequence of subword tokens, we cannot reliably infer where spaces should be inserted. A naive detokenizer that concatenates tokens typically loses word boundaries.
Therefor, by extending the vocabulary with a dedicated word-start marker ("_") and prefixing each word with this marker during encoding, detokenization becomes straightforward: concatenating tokens and replacing "_" by a space reconstructs word boundaries reliably. 